In [1]:
R.utils::gcDLLs() 
library(SingleCellExperiment)
require(scater)
require(Seurat)
require(mclust)
require(dplyr)
require(SC3)
require(scran)
require(biomaRt)
set.seed(1234567)

named list()

Loading required package: SummarizedExperiment
Loading required package: GenomicRanges
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsp

In [2]:
# Set the raw data and save/load it
umi <- NULL
generateit = FALSE

if (is.null(umi) && generateit){
    counts_raw <- read.table("/extra/sebastian_arnolds/analysis/trim/counts_matrix.tsv",sep="\t")
    barcodes_vector <- as.vector(
        read.table("/extra/sebastian_arnolds/analysis/celseq_barcodes_raw_all.txt")[, 1]
    )
    count_matrix <- as.matrix(counts_raw[, barcodes_vector])

    # Change labels for genes of interest
    name_map = list(
        ENSMUSG00000062327 = "Brachyury",
        ENSMUSG00000032446 = "Eomes",
        ENSMUSG00000030544 = "Mesp1",
        ENSMUSG00000024406 = "Pou5f1",
        ENSMUSG00000012396 = "nanog",
        ENSMUSG00000096014 = "Sox1",
        ENSMUSG00000074637 = "Sox2",
        ENSMUSG00000090125 = "Pou3f1",
        ENSMUSG00000060206 = "zfp462",
        ENSMUSG00000031297 = "slc7a3"
    )

    for (i in 1:length(name_map)){
        old_gene_name <- names(name_map[i])
        new_gene_name <- name_map[[i]]
        #message(old_gene_name, "→", new_gene_name)
        rownames(count_matrix)[which(rownames(count_matrix) == old_gene_name)] <- new_gene_name
    }

    saveRDS(count_matrix, "seurat_matrix.rds")
    umi <- count_matrix

} else {
    umi <- as.matrix(readRDS("seurat_matrix.rds"))
}

In [3]:
# Quick info
#message(sprintf("%.0f cells and %.0f genes", dim(umi)[2], dim(umi)[1]))

umi <- getBMFeatureAnnos(
    umi,
    filters = "ensembl_gene_id", 
    attributes = c("ensembl_gene_id", "external_gene_name"), 
    feature_symbol = "mgi_symbol",
    feature_id = "ensembl_gene_id",
    biomart = "ENSEMBL_MART_ENSEMBL",
    dataset = "mmusculus_gene_ensembl", 
    host = "www.ensembl.org"
)

ERROR: Error in dyn.load(file, DLLpath = DLLpath, ...): unable to load shared object '/usr/lib/R/library/curl/libs/curl.so':
  `maximal number of DLLs reached...
